In [41]:
import random as rd 
import numpy as np
import pprint 
pp = pprint.PrettyPrinter()

# Params to tune:

- `number_of_iterations`
- `n` (population size)
- `penalty weight` (this is a int scaller in the Eval func)
- `mutation_severity` and `threshold` (scallars in the Mutation func)

In [42]:
# Init variables, list, etc. 
limit: int = 25
mass: list[int] = [1,3,5,8,4,3,4,6,7,10] # total of 10 items
price:list[int] = [1,2,3,4,5,6,7,8,9,10]
id: list[str] = ['item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7', 'item_8', 'item_9', 'item_10']
number_of_iterations: int = 6000

#init population 
n: int = 500
population =[[None for j in range(len(id))] for i in range(n)]

In [43]:
# init population
for i in range(n):
    for j in range(len(id)):
        population[i][j] = rd.randint(0,1)

pp.pprint(population)

[[0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 0, 1, 1],
 [0, 1, 1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
 [1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 1, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 0, 1, 0],
 [0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
 [1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 1, 1, 0, 1, 1],
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 1],
 [0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [1, 1, 0, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
 [1, 1, 0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 0,

In [44]:
def Eval(population: list, id, mass, price, limit):
    weight = [0]*len(population)
    reward = [0]*len(population)
    for index in range(len(population)):
        for item in range(len(id)):
            weight[index] += mass[item]*population[index][item]
            reward[index] += price[item]*population[index][item]
        
        # If limit is exceeded deduct x amount of points from the reward
        if(weight[index]) > limit:
           delta = weight[index] - limit
           reward[index] -= 3 * delta
           if reward[index] < 0: # Prevent reward from becoming <0 (this could happen in some edge case)
               reward[index] = 0 
            
    return weight, reward

# For debugging
# weight, reward =(Eval(population=population, id=id, mass=mass, price=price, limit=limit))
# print(weight)
# print(reward)



In [45]:
def Mutation(population, n, mutation_severity=5, threshold=50): # Note that if threshold / mutation_severity < 10 no mutations will happen
    for index in range(len(population)):
        for item in range(n):
            if rd.randint(0,10) * mutation_severity >= threshold:
                if population[index][item] == 0:
                    population[index][item] = 1
                else:
                    population[index][item] = 0
    return population

In [46]:
def CrossOver(sorted_genome: list[int]):
    genome1, genome2 = sorted_genome[0], sorted_genome[1]
    pt = rd.randint(1, len(genome1)-2)
    g1 = genome1[:pt] + genome2[pt:]
    g2 = genome1[pt:] + genome2[:pt]
    return g1, g2

In [47]:
def NewGen(population, reward):
    sorted_indices = sorted(list(range(len(population))), key=lambda x: reward[x], reverse=True)
    sorted_genome = [population[i] for i in sorted_indices]

    pi = np.array([reward[i] / sum(reward) for i in sorted_indices])
    pi /= pi.sum()

    # generate a new population
    new_gen = []
    for i in range(len(population)-2): # create n-2 random populations
        new_gen.append(population[np.random.choice(len(population), p=pi)])

    # generate two crossover of the previous two best genome
    new_gen.append(CrossOver(sorted_genome)[0])
    new_gen.append(CrossOver(sorted_genome)[1])
    
    # Introduce some mutations
    new_pop = Mutation(new_gen, len(population[i]))
    return new_pop

In [48]:
# Run the simulation
for i in range(number_of_iterations):
    weight, reward = Eval(population, id, mass, price, limit)
    population = NewGen(population, reward)
    pp.pprint(f'Current itteration: {i}, and Reward: {max(reward)}, and Weight: {weight[reward.index(max(reward))]}')

# Get max reward genome 
optimal_genome = population[reward.index(max(reward))]

# Extract the data from the optimum solution (max reward)
index = 0
items = []
weight = 0
for i in optimal_genome:
    if i == 1:
        items.append(id[index])
        weight += mass[index]
    index += 1
 
# pp.pprint(f'Final rewards are: {reward}')
pp.pprint(f'Optimal Genome is: {optimal_genome}')
pp.pprint(f'Optimal Objects to Take: {items}')
pp.pprint(f'Weight of solution: {weight}')
print(f'Max weight is: {limit}')
pp.pprint(f'Reward of Optimimum {max(reward)}')

# pp.pprint(population)

'Current itteration: 0, and Reward: 33, and Weight: 25'
'Current itteration: 1, and Reward: 33, and Weight: 24'
'Current itteration: 2, and Reward: 33, and Weight: 24'
'Current itteration: 3, and Reward: 33, and Weight: 24'
'Current itteration: 4, and Reward: 33, and Weight: 24'
'Current itteration: 5, and Reward: 33, and Weight: 24'
'Current itteration: 6, and Reward: 33, and Weight: 24'
'Current itteration: 7, and Reward: 33, and Weight: 24'
'Current itteration: 8, and Reward: 33, and Weight: 24'
'Current itteration: 9, and Reward: 33, and Weight: 24'
'Current itteration: 10, and Reward: 33, and Weight: 24'
'Current itteration: 11, and Reward: 33, and Weight: 24'
'Current itteration: 12, and Reward: 33, and Weight: 24'
'Current itteration: 13, and Reward: 33, and Weight: 24'
'Current itteration: 14, and Reward: 33, and Weight: 24'
'Current itteration: 15, and Reward: 33, and Weight: 24'
'Current itteration: 16, and Reward: 33, and Weight: 24'
'Current itteration: 17, and Reward: 33, 